In [1]:
import os
import sys
import uuid
import cv2
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 300

In [2]:
nb_dir = '/'.join(os.getcwd().split('/')[:-1])
sys.path.append(nb_dir)
sys.path.append(os.path.split(nb_dir)[0])

import config
import src.utilities.app_context as app_context
app_context.init()

#path to craft model, weight can be pulled form the production bracnh of repo
config.CRAFT_MODEL_PATH= nb_dir + '/utilities/craft_pytorch/model/craft_mlt_25k.pth'
config.CRAFT_REFINE_MODEL_PATH = nb_dir + '/utilities/craft_pytorch/model/craft_refiner_CTW1500.pth'

from src.services.main import TextDetection

/home/naresh/ds-env2/lib/python3.6/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /pytorch/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


Loading weights of refiner from checkpoint (./model/craft_refiner_CTW1500.pth)
[2021-07-20 12:05:18,717] {geos.py:73} MainThread DEBUG in geos: Found GEOS DLL: <CDLL '/home/naresh/ds-env2/lib/python3.6/site-packages/shapely/.libs/libgeos_c-a68605fd.so.1.13.1', handle a8586f0 at 0x7fae37f3bba8>, using it.
[2021-07-20 12:05:18,777] {geos.py:38} MainThread DEBUG in geos: Trying `CDLL(libc.so.6)`
[2021-07-20 12:05:18,782] {geos.py:55} MainThread DEBUG in geos: Library path: 'libc.so.6'
[2021-07-20 12:05:18,785] {geos.py:56} MainThread DEBUG in geos: DLL: <CDLL 'libc.so.6', handle 7faed0887000 at 0x7fae37f3be10>


In [3]:
def draw_region(page_path,corrds,color= (255,0,0),thickness=5, save=False):
    
    if type(page_path) == str :  
        image = cv2.imread(page_path) 
    else :
        image = page_path
    for region in corrds :
            ground = region['boundingBox']['vertices']
            #start_point  = (ground[0]['x'],ground[0]['y'])
            #end_point = (ground[2]['x'],  ground[2]['y'])
            #cv2.rectangle(image, start_point, end_point, color,thickness)
            pts = []
            for pt in ground:
                pts.append([pt['x'] ,pt['y']])
            
            cv2.polylines(image, [np.array(pts)],True, color, thickness)
    plt.imshow(image)
    if save:
        cv2.imwrite(str(uuid.uuid1()) + '.png' , image)
        
    #return image


In [ ]:
base_dir = '/home/dhiraj/Documents/Anuwad/anuvaad/anuvaad-etl/anuvaad-extractor/block-merger/src/notebooks/sample-data/input'

filename    = 'ncert.pdf'
file_format = 'PDF'
language    =  'en'

In [ ]:
app_context.application_context = {
  "inputs": [
    {
      "file": {
        "identifier": "string",
        "name": filename,
        "format": file_format
      },
      "config": {
        "OCR": {
          "option": "HIGH_ACCURACY",
          "language": language
        }
      }
    }
  ]
}

In [ ]:
resp = TextDetection(app_context,base_dir)

In [ ]:
file_index = 0
page_index =0
to_check  = 'lines'

resp['rsp']['outputs'][file_index]['page_info']#[page_index][to_check]

In [ ]:
file_index = 0
page_index =0
to_check  = 'words'

regions    = resp['rsp']['outputs'][file_index]['pages'][page_index][to_check]
page_path = resp['rsp']['outputs'][file_index]['page_info'][page_index]


draw_region(page_path,regions)

In [ ]:
page_path

In [ ]:
page_image